<a href="https://colab.research.google.com/github/alinnman/lineofsight/blob/main/lighthouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sample for line of sight
This sample calculates the line of sight from a lighthouse to two observers located at sea, with different elevations above the sea surface. All **heights** are specified in meters. Calculations are made without and with refraction. For refraction **air pressure** (kPa), **temperature** (degrees Celsius) and **temperature gradient** (degrees Celsius / meter) must be entered. Default values are pre-set.

A calculation for an observer at sea level is also made, and a check is made to see if the parabola approximation ("8 inches per miles squared") is correct or not.

Instructions for use: Fill in the parameters and **Press Ctrl-F9 or the arrow button below** to calculate the result which will be presented in the bottom cell. <br/>Allow some time for the first execution (Google Colab needs some time to start up).

*Tip: To simulate severe refraction set the temperature gradient to 0.1*

In [ ]:
from math import sqrt, pi, atan2

# @markdown ### Enter parameters:
LIGHTHOUSE_HEIGHT = 56 # @param {"type":"integer"}
OBSERVER_HEIGHT_1 = 4 # @param {"type":"integer"}
OBSERVER_HEIGHT_2 = 40 # @param {"type":"integer"}
PRESSURE = 101 # @param {"type":"integer"}
TEMPERATURE = 20 # @param {"type":"integer"}
TEMP_GRADIENT = -0.01 # @param {"type":"number"}

EARTH_CIRCUMFERENCE_EQUATORIAL = 40075.017
EARTH_CIRCUMFERENCE_MERIDIONAL = 40007.86
EARTH_CIRCUMFERENCE = (EARTH_CIRCUMFERENCE_EQUATORIAL + EARTH_CIRCUMFERENCE_MERIDIONAL) / 2
EARTH_RADIUS = EARTH_CIRCUMFERENCE / (2 * pi)

R = EARTH_RADIUS*1000

def visibility_limit (R : float, h1 : float, h2 : float) -> float:
  ''' Geometry for line-of-sight '''
  x1a = sqrt (((R + h1)**2) - R**2)
  x1 = atan2 (x1a, R) * R
  x2a = sqrt (((R + h2)**2) - R**2)
  x2 = atan2 (x2a, R) * R
  return x1 + x2

DT_DH = TEMP_GRADIENT # Temperature shift with increasing elevation
K_FACTOR = 503*(PRESSURE*10)*(1/((TEMPERATURE+273)**2))*(0.0343 + DT_DH)
Ra = R / (1 - K_FACTOR) # This is the radius adjusted for refraction
# See https://en.wikipedia.org/wiki/Atmospheric_refraction#cite_note-Hirt2010-28

h1 = LIGHTHOUSE_HEIGHT # Height of lighthouse

print ("HEIGHT of LIGHTHOUSE = " + str(h1) + " meters")

for h2 in [0, OBSERVER_HEIGHT_1, OBSERVER_HEIGHT_2]: # Try some different observer elevations

  print ("")
  print ("==== CALCULATING for OBSERVER HEIGHT = " + str(h2) + " meters")

  ## No refraction

  distance = visibility_limit (R, h1, h2)
  print ("Lighthouse line of sight stretches = " + str(round(distance)) + " meters")

  # For zero elevation (observer at sea level) we can double-check with the parabola approximation
  # NOTE: The parabola approximation is useless if observer is above sea level!
  if h2 == 0:
    print ("Checking the 8 inches per miles squared approximation")
    MILES_PER_KM = 0.621371
    miles = (distance/1000) * MILES_PER_KM
    inches = 8 * (miles**2)
    INCHES_PER_METER = 39.3701
    meters_from_inches = inches / INCHES_PER_METER
    print ("Parabola approximation of lighthouse height = " + str(round(meters_from_inches,5)) + " meters")
    print (".. which is " + \
           str(abs(round(((meters_from_inches - h1) / h1),6)*100)) + \
           " % off the real value")

  ## Now adjust for refraction

  distance = visibility_limit (Ra, h1, h2)

  print ("Lighthouse line of sight stretches with refraction = " + str(round(distance)) + " meters")

HEIGHT of LIGHTHOUSE = 56 meters

==== CALCULATING for OBSERVER HEIGHT = 0 meters
Lighthouse line of sight stretches = 26716 meters
Checking the 8 inches per miles squared approximation
Parabola approximation of lighthouse height = 55.99769 meters
.. which is 0.0041 % off the real value
Lighthouse line of sight stretches with refraction = 28873 meters

==== CALCULATING for OBSERVER HEIGHT = 4 meters
Lighthouse line of sight stretches = 33856 meters
Lighthouse line of sight stretches with refraction = 36589 meters

==== CALCULATING for OBSERVER HEIGHT = 40 meters
Lighthouse line of sight stretches = 49295 meters
Lighthouse line of sight stretches with refraction = 53274 meters
